In [ ]:
import sys
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
sys.argv[1]='SC_Ura'
from aux import *


##Clear Memory 
tf.reset_default_graph()
tf.keras.backend.clear_session()
gc.collect()
##


NUM_GPU = len(get_available_gpus())
if(NUM_GPU>0) :
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

print(tf.__version__)
print(keras.__version__)
if sys.argv[1]=='Glu' :
    tpu_grpc_url = TPUClusterResolver(tpu=['edv-tpu3'] , zone='us-central1-a').get_master()
if(sys.argv[1] == 'SC_Ura') : 
    tpu_grpc_url = TPUClusterResolver(tpu=['edv-tpu1'] , zone='us-central1-a').get_master()

#sys.argv[1] appended to beginning of path
dir_path=os.path.join('..','..','data',sys.argv[1])
model_path=os.path.join(dir_path,"fitness_function.h5")


###
model_conditions = sys.argv[1]






## Load the data matrix
with h5py.File(join(dir_path,'_trX.h5'), 'r') as hf:
    _trX = hf['_trX'][:]

with h5py.File(join(dir_path,'_trY.h5'), 'r') as hf:
    _trY = hf['_trY'][:]

with h5py.File(join(dir_path,'_vaX.h5'), 'r') as hf:
    _vaX = hf['_vaX'][:]

with h5py.File(join(dir_path,'_vaY.h5'), 'r') as hf:
    _vaY = hf['_vaY'][:]

with h5py.File(join(dir_path,'_teX.h5'), 'r') as hf:
    _teX = hf['_teX'][:]

with h5py.File(join(dir_path,'_teY.h5'), 'r') as hf:
    _teY = hf['_teY'][:]


_trX.shape , _trY.shape , _vaX.shape , _vaY.shape  , _teX.shape , _teY.shape



trX = _trX #np.concatenate((_trX, _trX_rc), axis = 1) #np.squeeze((_trX))#
vaX = _vaX # np.concatenate((_vaX, _vaX_rc), axis = 1) #np.squeeze((_vaX))#
teX = _teX # np.concatenate((_teX, _teX_rc), axis = 1)#np.squeeze((_teX))#



## Load the scaler function (scaler was trained on the synthesized data
scaler = sklearn.externals.joblib.load(join(dir_path,'scaler.save')) 




vaY = (scaler.transform(_vaY.reshape(1, -1))).reshape(_vaY.shape) #_vaY#
trY = (scaler.transform(_trY.reshape(1, -1))).reshape(_trY.shape) #_trY#
teY = (scaler.transform(_teY.reshape(1, -1))).reshape(_teY.shape) #_teY#


### If using generator, have a smaller val set for faster evaluation
if 0 : 
    s_trX = np.vstack((trX , vaX))
    s_trY = np.vstack((trY , vaY))
    trX = s_trX[100000:,:]
    trY = s_trY[100000:,:]
    vaX = s_trX[0:100000,:]
    vaY = s_trY[0:100000,:]

print(trX.shape , trY.shape , vaX.shape , vaY.shape  , _teX.shape , _teY.shape)

input_shape = trX.shape



model_params = {
    'n_val_epoch' : 1000,
    'epochs' : 100,
    'batch_size': int(1024*1), # int(1024*3) , #64*55 fits , #best batch size is 1024
    'l1_weight': 0,#1e-6#1e-7#0.01 # l1 should always be zero
    'l2_weight': 1e-4,#1e-7#0.01
    'motif_conv_hidden': 256,
    'conv_hidden': 128,
    'n_hidden': 128, #128
    'n_heads': 8,
    'conv_width_motif':30, ##30bp for yeast is required for capturing all motifs 
    'dropout_rate': 0.2,
    'lr':0.001,
    'add_cooperativity_layer': True,
    'n_aux_layers': 2,
    'n_attention_layers':2,
    'attention_dropout_rate' : 0.2,
    'device_type' : 'tpu', #'tpu'/'gpu'/'cpu'
    'input_shape' : input_shape,
    'loss' : 'mean_squared_error'}
epochs = model_params['epochs']  
batch_size =  model_params['batch_size']
n_val_epoch =  model_params['n_val_epoch']
epochs = model_params['epochs']

### Save model params as csv
w = csv.writer(open(os.path.join(dir_path,'model_params.csv'), "w"))
for key, val in model_params.items():
    w.writerow([key, val])

### Save model params as pickle
f = open(os.path.join(dir_path,'model_params.pkl'),"wb")
pickle.dump(model_params,f)
f.close()





model=fitness_function_model(model_params)
if 1 :
    
    
    model.summary()
    #plot_model(model, show_shapes = 1 , show_layer_names = 1 ,to_file='model.png')

    #'Nadam'

    if(NUM_GPU > 1) : 
        model = multi_gpu_model(model, gpus=NUM_GPU, cpu_merge=True, cpu_relocation=False)


    if(tpu_grpc_url) : 
        model = tf.contrib.tpu.keras_to_tpu_model(model,
            strategy=tf.contrib.tpu.TPUDistributionStrategy(
                tf.contrib.cluster_resolver.TPUClusterResolver(tpu_grpc_url)))




### Randomly select dataset for the generator for the common model
class DataGenerator(keras.utils.Sequence):
    #https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly


    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, shuffle=True ):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        if self.shuffle == True:
            return int(np.floor(len(self.list_IDs) / self.batch_size))
        else:
            return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        if self.shuffle == True:
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        else:
            indexes = self.indexes[index*self.batch_size:np.minimum(len(self.list_IDs),(index+1)*self.batch_size)]    

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        batch_X , batch_Y = self.__data_generation(list_IDs_temp)
        return  (batch_X , batch_Y)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp ):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_X = trX[list_IDs_temp]
        batch_Y = trY[list_IDs_temp]


        return batch_X , batch_Y
    

if 0 : 
    params_train = {
              'batch_size': batch_size,
              'shuffle': True,
              }
    len_list_IDs = trX.shape[0]

    # Datasets
    partition = {'train': np.arange(len_list_IDs) }
    training_generator = DataGenerator(partition['train'], **params_train)
    example_batch = training_generator[0]


class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
history = LossHistory()

csvlogger = keras.callbacks.CSVLogger(os.path.join(dir_path,"loss_history.tsv"), separator='\t', append=False)
tensorboard = keras.callbacks.TensorBoard(log_dir=os.path.join(dir_path,"logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")),update_freq='batch', histogram_freq=0, batch_size=batch_size, write_graph=True, 
                            write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, 
                            embeddings_metadata=None, embeddings_data=None)


checkpoint = keras.callbacks.ModelCheckpoint(model_path, monitor='val_r_square', 
                                             verbose=1, save_best_only=True, mode='max')

# check 5 epochs
early_stop = keras.callbacks.EarlyStopping(monitor='val_r_square', patience=10, mode='max') 

callbacks_list = [checkpoint, early_stop, history , tensorboard , csvlogger]


model.fit(trX, trY, validation_data = (vaX, vaY),
          batch_size=batch_size  , epochs=epochs , callbacks=callbacks_list )

#model.fit_generator(training_generator, validation_data = (vaX, vaY),
#epochs=epochs , steps_per_epoch = int(trX.shape[0]/(batch_size*n_val_epoch)) , callbacks=callbacks_list )




if 0:
    model = load_model('keras_model.h5' , custom_objects={
        'MultiHeadAttention' : MultiHeadAttention , 
        'FeedForward' : FeedForward,
        'correlation_coefficient' : correlation_coefficient,
        'LayerNormalization' : LayerNormalization,
        'rc_Conv1D' : rc_Conv1D})







